## Statistical arguement against chance alignment
Similar to Sec 4.5 in Correia et al. 2006 (https://www.aanda.org/articles/aa/pdf/2006/45/aa5545-06.pdf)

I use Gaia EDR3 for this because KIC 8462852 B does not have proper motion in Gaia DR2

In [1]:
from astroquery.gaia import Gaia
import numpy as np
import astropy.units as u

# Boyajian's star's source id:
A_source_id = 2081900940499099136
B_source_id = 2081900944807842560

# Get Boyajian's star Gaia data:
job = Gaia.launch_job("SELECT * FROM gaiaedr3.gaia_source WHERE source_id = "+str(A_source_id))
a = job.get_results()

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [21]:
# Define a function to query Gaia for objects within radius and compute probability
# of observing a chance alignment
def get_prob(radius, catalog = 'gaiaedr3.gaia_source'):
    ''' For a given search radius, return the probability of a chance alignment of an object with similar
    parallax and proper motion to KIC 8462852 A within 2".  A very low probability means it is highly unlikely
    for KIC 8462852 B to be a chance alignment given the neighborhood it's in.
    
    Args:
        radius (flt): search radius.  Must be an astropy unit angle, such as u.deg or u.arcsec.
        catalog (str): set the catalog to query.  Default = Gaia EDR3.
        
    Returns:
        flt: probability of chance alignment of similar object
        int: number of similiar objects within search radius
        flt: surface density of similar objects; astropy unit object
    '''
    # make sure radius is in degrees:
    radius = radius.to(u.deg)
    # Define the query string:
    # KIC 8462852 ra/dec: (301.564, 44.4568)
    # KIC parallax = 2.25 mas -> parallax +/- 0.025 = 2.24,2.19
    search_string = "SELECT DISTANCE( \
          POINT('ICRS', ra, dec), \
          POINT('ICRS', 301.564, 44.4568)) AS dist, * \
        FROM "+catalog+" \
        WHERE 1=CONTAINS( POINT('ICRS',301.564, 44.4568), CIRCLE('ICRS',ra, dec, "+str(radius.value)+")) \
        AND sqrt(power(pmra - "+str(a['pmra'][0])+", 2) + power(pmdec - "+str(a['pmdec'][0])+", 2)) < 0.6 \
        AND parallax < 2.24  AND parallax > 2.19"
    # Perform query:
    job = Gaia.launch_job(search_string)
    g = job.get_results()
    N_obj = len(g)
    Sigma = N_obj/(np.pi*(radius.to(u.arcsec))**2)
    P = 1 - np.exp(-np.pi*Sigma*(2*u.arcsec**2))
    return P, N_obj, Sigma

# Query within 30 degrees:
P, N, Sigma = get_prob(30*u.deg)
P, N, Sigma

(<Quantity 2.40054867e-08>, 140, <Quantity 3.82059191e-09 1 / arcsec2>)

## Statistical arguement using binary statistics

#### Deacon et al. 2016 

But modified.  Deacon was working with color and magnitudes as mass estimates, and statistics for the entire binary population to decide if objects were binaries or not.  In this case I don't need to care about the entire binary population because I only care about one specific binary.  So I only need to worry about occurence rates of binaries for the bin of parameter space that my potential binary falls into.  The parameters here are: pm in ra & dec, Gaia Gmag, parallax, and area on the sky.  The probability is then a likelihood ratio of the likelihood of finding a binary at that bin of parameter space ($\phi_c$) over the likelihood of finding a binary or field star ($\phi_f$) in that bin.  The probability then is A2 in Deacon: $$P = \frac{\phi_c}{\phi_c + \phi_f}$$ and P$\approx$1 means it is much much more likely to be a companion than a field star.

### phi_f:

Query Gaia EDR3 to find the number of objects within an area of sky that match the companion in each bin of parameter space.

The units of $\phi_f$ will then be $objects \,(mas\, yr{^-1})^{-2} \,mas^{-1} \,arcsec^{-2}\,mag^{-1}$

In [3]:
from astroquery.gaia import Gaia
import numpy as np
import astropy.units as u

# Boyajian's star's source id:
A_source_id = 2081900940499099136
B_source_id = 2081900944807842560

# Get Boyajian's star Gaia data:
job = Gaia.launch_job("SELECT * FROM gaiaedr3.gaia_source WHERE source_id = "+str(A_source_id))
a = job.get_results()

job = Gaia.launch_job("SELECT * FROM gaiaedr3.gaia_source WHERE source_id = "+str(B_source_id))
b = job.get_results()

In [11]:
def get_phif(radius, catalog = 'gaiaedr3.gaia_source'):
    ''' For a given search radius, return the density of objects similar in pm, magnitude, and parallax
    to KIC 8462852 B
    
    Args:
        radius (flt): search radius.  Must be an astropy unit angle, such as u.deg or u.arcsec.
        catalog (str): set the catalog to query.  Default = Gaia EDR3.
        
    Returns:
        flt: phi_f, density of simiar objects in similar objects per arcsec^2
        table: similar objects
        int: number of similiar objects within search radius
        flt: size of search area in arcsec^2
    '''
    search_radius = radius.to(u.deg)
    # Slightly different search bins than above due to different considerations. This is a search
    # within :
    #   1 mag of B's magnitude (+/- 0.5 mag)
    #   1 mas/yr of A's proper motion in RA/DEC (+/- 0.5 mas/yr)
    #   0.5 mas of A's parallax (+/- 0.25 mas)
    search_string = "SELECT DISTANCE(\
          POINT('ICRS', ra, dec), \
          POINT('ICRS', 301.564, 44.4568)) AS dist, * \
        FROM FROM "+catalog+" \
        WHERE 1=CONTAINS(\
          POINT('ICRS',301.564, 44.4568), \
          CIRCLE('ICRS',ra, dec, "+str(search_radius.value)+")) \
        AND phot_g_mean_mag>=17.102833 AND phot_g_mean_mag<=18.102833 \
        AND pmra>=-10.8748 AND pmra<=-9.87484 \
        AND pmdec>=-10.773 AND pmdec<=-9.77311 \
        AND parallax>=2 AND parallax <=2.5 \
        ORDER BY dist ASC"
    # Launch query:
    job = Gaia.launch_job(search_string)
    k = job.get_results()
    k['dist_arcsec'] = k['dist']*u.deg.to(u.arcsec)
    
    # cut out potential binaries by removing closest objects to primary:
    max_sep_for_a_binary = 50000 # AU
    # convert to angle:
    a_distance = 451 #pc
    max_sep_for_a_binary = max_sep_for_a_binary / a_distance # arcsec
    field_stars = k[np.where(k['dist_arcsec']>max_sep_for_a_binary)[0]]
    
    # area of field stars search region:
    area_max = np.pi*(((search_radius.to(u.arcsec)))**2)
    area_min = (np.pi*(max_sep_for_a_binary*u.arcsec)**2)
    field_area = area_max-area_min
    
    # N_objects is x2 because the plx bin size was 1/2, so it's divided by the bin size
    N_obj = len(field_stars)*2
    
    phi_f = N_obj / field_area
    
    return phi_f, field_stars, N_obj , field_area 

### phi_c

Because we're not considering the distribution as a function of separation across the whole binary population, we can modify Deacon A3 to get the likelihood of finding a companion just in our secondary's bin of parameter space:
$$ \phi_c = c_s \times \big[\frac{ e^{ \frac{-\Delta\mu^2}{2\sigma^2_{\mu}}}} {2\pi\sigma^2_\mu}\big] \times \big[\frac{ e^{ \frac{-\Delta p^2}{2\sigma^2_{p}}}} {\sqrt{2\pi}\sigma_p}\big]$$

where p is the parallax.  So this gives units of $(mas\, yr^-1)^{-2} plx^{-1}$.  $c_s$ is then the modified normailzation factor which needs to be the occurrence rate of binaries in the same magnitude bin and in an area of the sky, giving units of $objects \,(mas\, yr{^-1})^{-2} \,mas^{-1} \,arcsec^{-2}\,mag^{-1}$ to match $\phi_f$

We used the binary demographics from Raghavan 2010, particularly Fig 13 and accompanying text, to generate occurence rates as a function of mass ratio and separation:

![alt text](Raghavan2010_binary_fraction.png "Title")

The approximate location of KIC 8462852 B is marked with the dot, the box is a representation of the parameter bins used for phi_c 


KIC 8462852 AB has a mass ratio of q=0.33.  Vertically the box is +/- 0.05 q, which corresponds to $\Delta$Gmag of 1.2 mags, using the mass and Gmag estimates of Mamajek table (Masses and M and m estimates written to the left).  

Horizontally the box is +/- 0.5 dex of separation (log(880) AU = 2.9 -> upper bound = 3.4 = 2512 AU = 5.6 arcsec, lower bound = 2.4 = 251.2 AU = 0.56 arcsec) so the area is a=$\pi$(5.6-0.56)$^2$ arcsec$^2$.  

Using Fig 13 of Raghavan 2010, ~10% of objects are binaries at 880 separation. Within the box, at the point, the occurence rate of binaries is ~0.025 per dex of sep per 0.25q.  So that makes the occurence rate (0.025/(0.25*0.1)) = 0.01 = 1%.  (times 0.1 because the box size is 0.1q)

So, put it all together:

$$ \phi_c = \big(0.01 \times \frac{1}{A} \times \frac{1}{\Delta mag}\big) \times \big[\frac{ e^{ \frac{-\Delta\mu^2}{2\sigma^2_{\mu}}}} {2\pi\sigma^2_\mu}\big] \times \big[\frac{ e^{ \frac{-\Delta \pi^2}{2\sigma^2_{\pi}}}} {\sqrt{2\pi}\sigma_\pi}\big] $$

which gives units of $objects \,(mas\, yr^{-1})^{-2} \,mas^{-1} \,arcsec^{-2}\,mag^{-1}$

In [23]:
# A small function for performing Monte Carlo simulations:
from tools import MonteCarloIt

def get_phic():
    ''' Get phi_c for KIC 8462852 B
    '''
    # Run a Monte Carlo Simulation to get uncertainties:
    # rel proper motion:
    dmu_ra = MonteCarloIt([b['pmra'],b['pmra_error']]) - MonteCarloIt([a['pmra'],a['pmra_error']])
    dmu_dec = MonteCarloIt([b['pmdec'],b['pmdec_error']]) - MonteCarloIt([a['pmdec'],a['pmdec_error']])
    dmu = np.sqrt(np.mean(dmu_ra)**2 + np.mean(dmu_dec)**2) #mas/yr
    sigma_mu = np.sqrt(np.std(dmu_ra)**2 + np.std(dmu_dec)**2)

    dplx = b['parallax'][0] - a['parallax'][0] # mas
    sigma_plx = np.sqrt(b['parallax_error'][0]**2 + a['parallax_error'][0]**2)
    # Magntiude bin:
    dmag = 1.2 # mags
    # Separation bin:
    area = np.pi*((5.6-0.56)**2) # arcsec^2

    A = (1/(2*np.pi*sigma_mu**2)) * np.exp(-dmu**2/(2*sigma_mu**2))
    B = (1/(np.sqrt(2*np.pi)*sigma_plx)) * np.exp(-dplx**2/(2*sigma_plx**2))
    C = 0.01 * (1/area) * ((1/dmag)*(1/1.2)) #<- to make the magnitude bin the same as it was for the field.

    phi_c = A * B * C
    return phi_c

In [22]:
def get_probability(radius, catalog = 'gaiaedr3.gaia_source'):
    ''' For a given search radius, return the probability that KIC 8462852 B is a binary companion.
    
    Args:
        radius (flt): search radius.  Must be an astropy unit angle, such as u.deg or u.arcsec.
        catalog (str): set the catalog to query.  Default = Gaia EDR3.
        
    Returns:
        flt: probability that KIC 8462852 B is a binary companion
    '''
    phi_f = get_phif(radius, catalog = catalog)
    phi_c = get_phic()
    P = phi_c / (phi_c+phi_f[0].value)
    return P, phi_c, phi_f[0], phi_f[2]

P, phi_c, phi_f, N_obj_phif = get_probability(30*u.deg)
print('Probability:',P)
print('phi_c',phi_c)
print('phi_f',phi_f)
print('N objects in field',N_obj_phif)

Probability: 0.9999716554765988
phi_c 0.0001270853637096497
phi_f 3.6022761704223967e-09 1 / arcsec2
N objects in field 132
